Lo que hemos visto en la sección anterior, es una forma más manual de hacer la normalización.

A continuación, condensaremos el conocimiento anterior.


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch


transform = transforms.ToTensor()

train = datasets.CIFAR10("./data", train=True, download=True, transform=transform)
test_train = datasets.CIFAR10("./data", train=False, download=True, transform=transform)

loader_train = DataLoader(train, batch_size=len(train), shuffle=True)
loader_test = DataLoader(test_train, batch_size=len(test_train), shuffle=True)

imgs, _ = next(iter(loader_train))             # [50000,3,32,32]
imgs = imgs.to(torch.float64)

mean = imgs.mean(dim=(0,2,3))
std  = imgs.std(dim=(0,2,3))
print(mean, std)


cuda
mean: tensor([0.4881, 0.4829, 0.4498])
std : tensor([0.2425, 0.2402, 0.2549])
torch.Size([128, 3, 32, 32]) torch.Size([128])
Mean (debería estar cerca de 0): tensor([0.0197, 0.0268, 0.0116])
Std (debería estar cerca de 1): tensor([1.0364, 1.0124, 1.0330])


| Parámetro    | Valor                                                      |
| ------------ | ---------------------------------------------------------- | 
| Mean ≈ -0.05 |  Correcto — centrado en 0 con ligera desviación por batch | 
| Std ≈ 1.0    |  Perfecto — escala normalizada                            |                        
